In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Energy.csv')
print(df.info())

In [ ]:
df = df.replace('Not Available', np.nan)
# rename score
df.rename(columns={'ENERGY STAR Score': 'score'}, inplace=True)
df = df[~df['score'].isnull()]
for c in list(df.columns):
    if 'ft²' in c or 'kBtu' in c or 'Metric Tons CO2e' in c or 'kWh' in c or 'therms' in c or 'gal' in c or 'Score' in c or 'score' in c:
        df[c] = df[c].astype(float)
print(df.info())

In [ ]:
mis_count = df.isnull().sum()
mis_percent = mis_count/len(df)
mis = pd.DataFrame({'count': mis_count, 'percent': mis_percent})
mis.sort_values(by='count', ascending=False, inplace=True)
print(mis)

In [ ]:
# drop whose percent > 50
mis_50 = mis[mis['percent'] > 0.5].index
print(mis_50)
df.drop(mis_50, axis=1, inplace=True)
print(df.info())

In [ ]:
print(len(df))

9642


In [ ]:
# Score distribution: the distribution is funny
sns.distplot(df['score'])
plt.title('score distribution')
plt.show()

In [ ]:
# Instead, look at EUI, which is objective; outliers
sns.distplot(df['Site EUI (kBtu/ft²)'])
plt.title('EUI distribution')
plt.show()
# there must be extreme values

In [ ]:
# box plot
sns.boxplot(y='Site EUI (kBtu/ft²)', data=df)
plt.show()

In [ ]:
# drop outliers
df['Site EUI (kBtu/ft²)'].fillna(df['Site EUI (kBtu/ft²)'].mean(), inplace=True)
print(df['Site EUI (kBtu/ft²)'].isnull().sum())
q25 = np.percentile(df['Site EUI (kBtu/ft²)'].values, 25)
q75 = np.percentile(df['Site EUI (kBtu/ft²)'].values, 75)
iqr = q75-q25
upper = q75 + 3*iqr
lower = q25 - 3*iqr
df = df[(df['Site EUI (kBtu/ft²)']<upper) & (df['Site EUI (kBtu/ft²)']>lower)]
sns.distplot(df['Site EUI (kBtu/ft²)'])
plt.show()
sns.boxplot(y='Site EUI (kBtu/ft²)', data=df)
plt.show()

In [ ]:
# longitude & latitude:
df.plot(kind='scatter', x='Longitude', y='Latitude', alpha=0.4, s=1, c='score', cmap=plt.get_cmap('jet'))
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('longitude & latitude  matters?')
plt.show()

In [ ]:
# Year built
# df.drop(df[df['Year Built'] < 1800].index, inplace=True)
# df.drop(df[df['Year Built'] > 2016].index, inplace=True)
year = df.groupby('Year Built')['score'].mean().sort_index()
print(list(year.index))
plt.bar(list(year.index), year.values)
plt.xlabel('year built')
plt.ylabel('mean score')
plt.title('mean score for each year')
plt.show()

In [ ]:
# house's type
UseType_counts = df['Largest Property Use Type'].value_counts()
UseType = UseType_counts[UseType_counts.values >= 50].index
print(UseType)
plt.figure(figsize=(24, 20))
for T in UseType:
    subset = df[df['Largest Property Use Type'] == T]
    sns.distplot(subset['score'], hist=False, label=T)
plt.xlabel('score')
plt.ylabel('distribute')
plt.title('Largest property use type influence')
plt.show()

In [ ]:
# borough
borough = df['Borough'].value_counts().sort_values(ascending=False).head(5).index
print(borough)
plt.figure(figsize=(12, 10))
for b in borough:
    subset = df[df['Borough'] == b]
    sns.distplot(subset['score'], hist=False, label=b)
plt.xlabel('score')
plt.ylabel('distribute')
plt.title('Borough influence')
plt.show()

In [ ]:
# Primary Property Type
PPType_counts = df['Primary Property Type - Self Selected'].value_counts()
PPType = PPType_counts[PPType_counts.values >= 50].index
plt.figure(figsize=(24, 20))
for T in PPType:
    subset = df[df['Primary Property Type - Self Selected'] == T]
    sns.distplot(subset['score'], hist=False, label=T)
plt.xlabel('score')
plt.ylabel('distribute')
plt.title('Primary Property type influence')
plt.show()

In [ ]:
# Water required 
for w in df['Water Required?'].unique():
    subset = df[df['Water Required?'] == w]
    sns.distplot(subset['score'], hist=False, label=w)
plt.xlabel('score')
plt.ylabel('distribute')
plt.title('Water required influence')
plt.show()

In [ ]:
col_dig = list(df.select_dtypes(['number']).columns)
print('digital features: ', col_dig)

digital features:  ['Order', 'Property Id', 'DOF Gross Floor Area', 'Largest Property Use Type - Gross Floor Area (ft²)', 'Year Built', 'Number of Buildings - Self-reported', 'Occupancy', 'score', 'Site EUI (kBtu/ft²)', 'Weather Normalized Site EUI (kBtu/ft²)', 'Weather Normalized Site Electricity Intensity (kWh/ft²)', 'Weather Normalized Site Natural Gas Intensity (therms/ft²)', 'Weather Normalized Source EUI (kBtu/ft²)', 'Natural Gas Use (kBtu)', 'Weather Normalized Site Natural Gas Use (therms)', 'Electricity Use - Grid Purchase (kBtu)', 'Weather Normalized Site Electricity (kWh)', 'Total GHG Emissions (Metric Tons CO2e)', 'Direct GHG Emissions (Metric Tons CO2e)', 'Indirect GHG Emissions (Metric Tons CO2e)', 'Property GFA - Self-Reported (ft²)', 'Water Use (All Water Sources) (kgal)', 'Water Intensity (All Water Sources) (gal/ft²)', 'Source EUI (kBtu/ft²)', 'Latitude', 'Longitude', 'Community Board', 'Council District', 'Census Tract']


In [ ]:
# corr matrix
corr = df.corr()
plt.figure(figsize=(48, 36))
sns.heatmap(corr, cmap='coolwarm')
plt.title('corr map')
plt.show()
print(corr['score'].sort_values())
cols = []
for col in list(corr[np.abs(corr['score']) > 0.5].index):
    if col in col_dig:
        cols.append(col)
cols.remove('score')
print('features highly related to score: ', cols)

In [ ]:
# grid plot those features
plt.figure(figsize=(48, 36))
grid = sns.PairGrid(df[cols])
grid.map_upper(plt.scatter)
grid.map_lower(sns.kdeplot)
grid.map_diag(plt.hist)
plt.title('relation of the feature highly related to score')
plt.show()

In [ ]:
# delete collinear digital features
threshold = 0.7
score = df['score']
df_dig = df[col_dig]
corr_dig = df_dig.corr()
col_drop = []
for i in range(1, len(col_dig)):
    for j in range(0, i):
        val = corr_dig.iloc[j, i]
        if val >= threshold:
            col_drop.append(col_dig[i])
col_drop = set(col_drop)
df_dig.drop(col_drop, axis=1, inplace=True)
print(df_dig.index)

Int64Index([   12,    18,    19,    20,    21,    22,    23,    30,    31,
               32,
            ...
            11736, 11737, 11738, 11739, 11740, 11741, 11742, 11743, 11744,
            11745],
           dtype='int64', length=9442)


In [ ]:
# imputation and scale
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(np.nan, 'mean')
Index = df_dig.index
df_dig = pd.DataFrame(imputer.fit_transform(df_dig))

robustScaler = RobustScaler()
df_dig = pd.DataFrame(robustScaler.fit_transform(df_dig.values), index=Index)
print(df_dig.index)
print(df_dig.shape)
print(df_dig.isnull().sum())


Int64Index([   12,    18,    19,    20,    21,    22,    23,    30,    31,
               32,
            ...
            11736, 11737, 11738, 11739, 11740, 11741, 11742, 11743, 11744,
            11745],
           dtype='int64', length=9442)
(9442, 17)
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
dtype: int64


In [ ]:
# one hot category features
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# category features are 'Largest Property Use Type', 'Borough', 'Primary Property Type - Self Selected',
l1 = pd.get_dummies(df['Largest Property Use Type'])
l2 = pd.get_dummies(df['Borough'])
l3 = pd.get_dummies(df['Primary Property Type - Self Selected'])
print(l1.isnull().sum())
print(l2.isnull().sum())
print(l3.isnull().sum())

Bank Branch                              0
Courthouse                               0
Distribution Center                      0
Financial Office                         0
Hospital (General Medical & Surgical)    0
Hotel                                    0
K-12 School                              0
Medical Office                           0
Multifamily Housing                      0
Non-Refrigerated Warehouse               0
Office                                   0
Parking                                  0
Refrigerated Warehouse                   0
Residence Hall/Dormitory                 0
Retail Store                             0
Senior Care Community                    0
Supermarket/Grocery Store                0
Wholesale Club/Supercenter               0
Worship Facility                         0
dtype: int64
Bronx            0
Brooklyn         0
Manhattan        0
Queens           0
Staten Island    0
dtype: int64
Bank Branch                              0
College/University 

In [ ]:
# concat
print(df_dig.index)
print(l1.index)
df_full = pd.concat([df_dig, l1, l2, l3, score], axis=1)
print(df_full.isnull().sum())
print(df_full.shape)

Int64Index([   12,    18,    19,    20,    21,    22,    23,    30,    31,
               32,
            ...
            11736, 11737, 11738, 11739, 11740, 11741, 11742, 11743, 11744,
            11745],
           dtype='int64', length=9442)
Int64Index([   12,    18,    19,    20,    21,    22,    23,    30,    31,
               32,
            ...
            11736, 11737, 11738, 11739, 11740, 11741, 11742, 11743, 11744,
            11745],
           dtype='int64', length=9442)
0                             0
1                             0
2                             0
3                             0
4                             0
                             ..
Senior Care Community         0
Supermarket/Grocery Store     0
Wholesale Club/Supercenter    0
Worship Facility              0
score                         0
Length: 66, dtype: int64
(9442, 66)


In [ ]:
# train test set
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_full, test_size=0.3)
train_x = train.drop('score', axis=1)
print(train_x.shape)
print(train_y.shape)
train_y = train['score']
test_x = test.drop('score', axis=1)
test_y = test['score']
train_x.to_csv('train_x.csv')
train_y.to_csv('train_y.csv')
test_x.to_csv('test_x.csv')
test_y.to_csv('test_y.csv')


(6609, 65)
(6609,)


In [ ]:
import pandas as pd
import numpy as np

# metric: MSE
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
# naive regression model:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

# grid search hyper parameters
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

# read train test set
train_x = pd.read_csv('train_x.csv').values
train_y = pd.read_csv('train_y.csv').values.reshape(-1,)
test_x = pd.read_csv('test_x.csv').values
test_y = pd.read_csv('test_y.csv').values.reshape(-1,)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)


parameter_space = {
    "n_estimators": [10, 15, 20, 25, 30],
    "criterion": ["mse", "mae"],
    "min_samples_leaf": [2, 4, 6],
    'min_samples_split': [2, 4, 6],
    'max_depth': [5, 8, 10, 12, 15]
}
forest = RandomForestRegressor()
grid = GridSearchCV(forest, parameter_space, cv=5, n_jobs=-1, verbose=1)
grid.fit(train_x, train_y)
rf = grid.best_estimator_

predict_y_rf = rf.predict(test_x)
import seaborn as sns
import matplotlib.pyplot as plt
sns.distplot(predict_y_rf, hist=False, label='predict')
sns.distplot(test_y, hist=False, label='True')
plt.show()

error = predict_y_rf - test_y
sns.distplot(error)
plt.show()

(6609, 64)
(6609,)
(2833, 64)
(2833,)
Fitting 5 folds for each of 450 candidates, totalling 2250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored